# Custom Loss
- In machine learning, a loss function, also known as a cost function or objective function, is a mathematical function that quantifies the difference between the predicted output of a model and the actual expected output. It represents the error or discrepancy between the model's predictions and the ground truth.
- Note that, loss is associated with a single example whereas cost is associated with a training batch (average of loss of each example of a batch)
- Ways to create custom loss functions in Keras
   - using python function that takes y_true and y_pred values

## 1. Function based custom loss

The first method involves creating a function that accepts inputs **y_true** and **y_pred**. 

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

2023-07-17 07:45:54.911014: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-17 07:45:54.955888: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-17 07:45:54.956803: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 07:45:55.992903: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [25]:
model = keras.Sequential([
    layers.Input((10)),
    layers.Dense(10, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                110       
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [54]:
x = tf.random.normal([10, 10])
y_true = tf.round(tf.random.uniform((10, 1), minval=0, maxval=1))
x.shape, y_true.shape

(TensorShape([10, 10]), TensorShape([10, 1]))

In [68]:
def custom_MSE(y_true, y_pred):
    return tf.math.reduce_mean(tf.square(y_true - y_pred), axis=-1)

In [64]:
model.compile(loss=custom_MSE, optimizer="adam")

In [73]:
model.fit(x, y_true , epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 7ms/step - loss: 0.2496
Epoch 2/10
1/1 [==============================] - 0s 5ms/step - loss: 0.2481
Epoch 3/10
1/1 [==============================] - 0s 4ms/step - loss: 0.2466
Epoch 4/10
1/1 [==============================] - 0s 9ms/step - loss: 0.2451
Epoch 5/10
1/1 [==============================] - 0s 6ms/step - loss: 0.2436
Epoch 6/10
1/1 [==============================] - 0s 10ms/step - loss: 0.2422
Epoch 7/10
1/1 [==============================] - 0s 5ms/step - loss: 0.2409
Epoch 8/10
1/1 [==============================] - 0s 7ms/step - loss: 0.2395
Epoch 9/10
1/1 [==============================] - 0s 6ms/step - loss: 0.2381
Epoch 10/10
1/1 [==============================] - 0s 7ms/step - loss: 0.2368


Loss function is working fine, you can see the loss value is reducing. 

## Subclassing based custom loss
If you need a loss function that takes in parameters beside **y_true** and **y_pred**, you can subclass the **tf.keras.losses.Loss** class and implement the following two methods:
- **\_\_init\_\_(self)**: accept parameters to pass during the call of your loss function
- **call(self, y_true, y_pred)**: use the targets (y_true) and the model predictions (y_pred) to compute the model's loss


In [74]:
class CustomMSE(keras.losses.Loss):
    def __init__(self, regularization_factor=0.1, name="custom_MSE"):
        super().__init__(name=name)
        self.regularization_factor = regularization_factor
    
    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred), axis=-1)
        reg = reg = tf.math.reduce_mean(tf.square(0.5 - y_pred), axis=-1)
        return mse + reg * self.regularization_factor

In [75]:
model = keras.Sequential([
    layers.Input((10)),
    layers.Dense(10, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(loss=CustomMSE(regularization_factor=0.2), optimizer="adam")

In [76]:
model.fit(x, y_true, epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 342ms/step - loss: 0.2909
Epoch 2/10
1/1 [==============================] - 0s 4ms/step - loss: 0.2894
Epoch 3/10
1/1 [==============================] - 0s 7ms/step - loss: 0.2879
Epoch 4/10
1/1 [==============================] - 0s 4ms/step - loss: 0.2864
Epoch 5/10
1/1 [==============================] - 0s 11ms/step - loss: 0.2849
Epoch 6/10
1/1 [==============================] - 0s 7ms/step - loss: 0.2834
Epoch 7/10
1/1 [==============================] - 0s 8ms/step - loss: 0.2819
Epoch 8/10
1/1 [==============================] - 0s 10ms/step - loss: 0.2805
Epoch 9/10
1/1 [==============================] - 0s 5ms/step - loss: 0.2790
Epoch 10/10
1/1 [==============================] - 0s 8ms/step - loss: 0.2775


## 3. Loss Function Wrapper bases custom loss
- Alternatively you could implement the loss function as a method, and use the **LossFunctionWrapper** to turn it into a class.
- This wrapper is a subclass of **tf.keras.losses.Loss** which handles the parsing of extra arguments by passing them to the **call()** and **config methods**.


The LossFunctionWrapper's __init__() method takes the following arguments:
- **fn**: The loss function to wrap, with signature fn(y_true, y_pred, **kwargs).
- **reduction**: Type of tf.keras.losses.Reduction to apply to loss.
- **name**: Optional name for the instance.
- Any other parameters will be passed to fn as kwargs through the call() method.


In [87]:
# tensorflow keras doesnot have LossFunctionWrapper
from keras import losses

In [83]:
model = keras.Sequential([
    layers.Input((10)),
    layers.Dense(10, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

In [84]:
def custom_MSE(y_true, y_pred, regularization_factor=0.1):
    mse = tf.math.reduce_mean(tf.square(y_true - y_pred), axis=-1)
    reg = tf.math.reduce_mean(tf.square(0.5 - y_pred), axis=-1)
    return mse + reg * regularization_factor

In [86]:
class WrappedCustomMSE(losses.LossFunctionWrapper):
    def __init__(self, 
        reduction=tf.keras.losses.Reduction.AUTO,
        name="custom_mse_with_regularization",
        regularization_factor=0.1,
    ):
        
        super().__init__(fn=custom_MSE,
                        reduction=reduction,
                        name=name,
                        regularization_factor=regularization_factor)

In [88]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=WrappedCustomMSE(regularization_factor=0.2, name="mse_custom_0_2"),
)

In [89]:
model.fit(x, y_true, epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 381ms/step - loss: 0.2725
Epoch 2/10
1/1 [==============================] - 0s 7ms/step - loss: 0.2708
Epoch 3/10
1/1 [==============================] - 0s 10ms/step - loss: 0.2692
Epoch 4/10
1/1 [==============================] - 0s 6ms/step - loss: 0.2675
Epoch 5/10
1/1 [==============================] - 0s 6ms/step - loss: 0.2659
Epoch 6/10
1/1 [==============================] - 0s 8ms/step - loss: 0.2643
Epoch 7/10
1/1 [==============================] - 0s 8ms/step - loss: 0.2626
Epoch 8/10
1/1 [==============================] - 0s 6ms/step - loss: 0.2611
Epoch 9/10
1/1 [==============================] - 0s 12ms/step - loss: 0.2596
Epoch 10/10
1/1 [==============================] - 0s 8ms/step - loss: 0.2580


## 4. Nasted function (not mentioned in the official documentation)
- Use subclassing approach insted of this as it is not mentioned in the official documentation

In [90]:
def custom_MSE(regularization_factor):
    def MSE(y_pred, y_true):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred), axis=-1)
        reg = tf.math.reduce_mean(tf.square(0.5 - y_pred), axis=-1)
        return mse + reg * regularization_factor
    return MSE

In [93]:
model = keras.Sequential([
    layers.Input((10)),
    layers.Dense(10, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=custom_MSE(regularization_factor=0.2)
)

In [94]:
model.fit(x, y_true, epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 303ms/step - loss: 0.3893
Epoch 2/10
1/1 [==============================] - 0s 6ms/step - loss: 0.3869
Epoch 3/10
1/1 [==============================] - 0s 11ms/step - loss: 0.3846
Epoch 4/10
1/1 [==============================] - 0s 6ms/step - loss: 0.3823
Epoch 5/10
1/1 [==============================] - 0s 6ms/step - loss: 0.3800
Epoch 6/10
1/1 [==============================] - 0s 8ms/step - loss: 0.3778
Epoch 7/10
1/1 [==============================] - 0s 5ms/step - loss: 0.3756
Epoch 8/10
1/1 [==============================] - 0s 9ms/step - loss: 0.3734
Epoch 9/10
1/1 [==============================] - 0s 8ms/step - loss: 0.3712
Epoch 10/10
1/1 [==============================] - 0s 7ms/step - loss: 0.3690
